In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from sklearn.datasets import load_diabetes

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = load_diabetes()

In [3]:
print(f'X Shape : {data.data.shape}')
print(f'Y Shape : {data.target.shape}')

X Shape : (442, 10)
Y Shape : (442,)


In [4]:
print(data['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [5]:
pd.DataFrame(data['data'], columns=data['feature_names']).describe().map(lambda x: f"{x:0.2f}")

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
count,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00
mean,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00
std,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
min,-0.11,-0.04,-0.09,-0.11,-0.13,-0.12,-0.10,-0.08,-0.13,-0.14
25%,-0.04,-0.04,-0.03,-0.04,-0.03,-0.03,-0.04,-0.04,-0.03,-0.03
50%,0.01,-0.04,-0.01,-0.01,-0.00,-0.00,-0.01,-0.00,-0.00,-0.00
75%,0.04,0.05,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03
max,0.11,0.05,0.17,0.13,0.15,0.20,0.18,0.19,0.13,0.14


In [6]:
Y = pd.DataFrame(data['target'], columns=['Y'])
X = pd.DataFrame(data['data'], columns=data['feature_names'])
# X = pd.get_dummies(X, columns=['sex'])

In [7]:
data = pd.read_csv('https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt', sep='\t')

In [8]:
Y = data['Y']
X = data.drop(columns=['Y']) 
X = pd.get_dummies(X, columns=['SEX'])

In [9]:
X

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_1,SEX_2
0,59,32.1,101.00,157,93.2,38.0,4.00,4.8598,87,False,True
1,48,21.6,87.00,183,103.2,70.0,3.00,3.8918,69,True,False
2,72,30.5,93.00,156,93.6,41.0,4.00,4.6728,85,False,True
3,24,25.3,84.00,198,131.4,40.0,5.00,4.8903,89,True,False
4,50,23.0,101.00,192,125.4,52.0,4.00,4.2905,80,True,False
...,...,...,...,...,...,...,...,...,...,...,...
437,60,28.2,112.00,185,113.8,42.0,4.00,4.9836,93,False,True
438,47,24.9,75.00,225,166.0,42.0,5.00,4.4427,102,False,True
439,60,24.9,99.67,162,106.6,43.0,3.77,4.1271,95,False,True
440,36,30.0,95.00,201,125.2,42.0,4.79,5.1299,85,True,False


In [10]:
idx = list(range(X.shape[0]))
train_idx, test_idx = train_test_split(idx, test_size=0.3, random_state=2023)

In [11]:
results = LinearRegression().fit(X.iloc[train_idx], Y.iloc[train_idx])
results

LinearRegression()

In [12]:
import scipy
from sklearn import metrics

def sse(clf, X, y):
    """
    standard squared error(residual)
    
    Params
    ----
    clf : linear model
    X : training data
    y : target value
    
    return
    ----
    float
    """
    y_hat = clf.predict(X)
    sse = np.sum((y_hat - y)**2)
    return sse/X.shape[0]

def adj_r2_score(clf, X, y):
    """
    adjusted r2 score
    
    Params
    ----
    clf : linear model
    X : training data
    y : target value
    
    return
    ----
    float
    """
    n = X.shape[0]
    p = X.shape[1]
    y_hat = clf.predict(X)
    r_squared = metrics.r2_score(y, y_hat)
    return 1-(1-r_squared)*(n-1)/(n-p-1)

def coef_se(clf, X, y):
    """
    standard error of beta coefficients
    
    Params
    ----
    clf : linear model
    X : training data
    y : target value
    
    return
    ----
    float
    """
    n = X.shape[0]
    X = X.map(lambda x: float(x))
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    # print(X1,"X1")
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X))* np.linalg.pinv(X1.T * X1)
    )
    # print(se_matrix,"se_matrix")
    # print(np.linalg.pinv(X1.T * X1),"np.linalg.inv(X1.T * X1)")
    return np.diagonal(se_matrix)

def coef_tval(clf, X, y):
    """Calculate t-statistic for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of t-statistic values.
    """
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_pval(clf, X, y):
    """Calculate p-values for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of p-values.
    """
    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def summary(clf, X, y, xlabels=None):
    """
    Output summary statistics for a fitted regression model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    xlabels : list, tuple
        The labels for the predictors.
    """
    # Check and/or make xlabels
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')
    # Make sure dims of xlabels matches dims of X
    if xlabels.shape[0] != ncols:
        raise AssertionError(
            "Dimension of xlabels {0} does not match "
            "X {1}.".format(xlabels.shape, X.shape))
    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate', 'Std. Error', 't value', 'p value']
    )
    try:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
    except Exception as e:
        coef_df['Estimate'] = np.concatenate(
            (
                np.round(np.array([clf.intercept_]), 6),
                np.round((clf.coef_), 6)
            ), axis = 1
    )[0,:]
    coef_df['Std. Error'] = np.round(coef_se(clf, X, y), 6)
    coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
    coef_df['p value'] = np.round(coef_pval(clf, X, y), 6)
    # Output results
    print('Coefficients:')
    print(coef_df.to_string(index=True))
    print('---')
    print('R-squared:  {0:.6f},    Adjusted R-squared:  {1:.6f},    MSE: {2:.1f}'.format(
        metrics.r2_score(y, clf.predict(X)), adj_r2_score(clf, X, y), sse(clf, X, y)))

In [13]:
summary(results, X.iloc[test_idx], Y.iloc[test_idx], xlabels=X.columns) 

Coefficients:
              Estimate            Std. Error         t value   p value
_intercept -353.422717  65.598292+ 0.000000j -5.3877+0.0000j  0.000000
AGE          -0.241046   0.393782+ 0.000000j -0.6121+0.0000j  0.541504
BMI           5.364734   1.269974+ 0.000000j  4.2243-0.0000j  0.000044
BP            0.973515   0.360440+ 0.000000j  2.7009-0.0000j  0.007822
S1           -1.128987   0.293810+ 0.000000j -3.8426+0.0000j  0.000188
S2            0.935342   0.352460+ 0.000000j  2.6538-0.0000j  0.008938
S3            0.295834   0.415883+ 0.000000j  0.7113-0.0000j  0.478129
S4            2.577375  10.098720+ 0.000000j  0.2552-0.0000j  0.798952
S5           72.840272  21.496297+ 0.000000j  3.3885-0.0000j  0.000927
S6            0.292290   0.382362+ 0.000000j  0.7644-0.0000j  0.445974
SEX_1        10.444984  20.017461+ 0.000000j  0.5218-0.0000j  0.602688
SEX_2       -10.444984  20.193080+ 0.000000j -0.5173+0.0000j  0.605843
---
R-squared:  0.530165,    Adjusted R-squared:  0.487453,    

In [14]:
scaler = MinMaxScaler().fit(X.iloc[train_idx])
X_scal = scaler.transform(X)
X_scal = pd.DataFrame(X_scal, columns=X.columns)

In [15]:
X_scal

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_1,SEX_2
0,0.666667,0.573840,0.565217,0.294118,0.297578,0.197368,0.318471,0.562217,0.439394,0.0,1.0
1,0.483333,0.130802,0.362319,0.421569,0.355248,0.618421,0.159236,0.222437,0.166667,1.0,0.0
2,0.883333,0.506329,0.449275,0.289216,0.299885,0.236842,0.318471,0.496578,0.409091,0.0,1.0
3,0.083333,0.286920,0.318841,0.495098,0.517878,0.223684,0.477707,0.572923,0.469697,1.0,0.0
4,0.516667,0.189873,0.565217,0.465686,0.483276,0.381579,0.318471,0.362385,0.333333,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.683333,0.409283,0.724638,0.431373,0.416378,0.250000,0.318471,0.605672,0.530303,0.0,1.0
438,0.466667,0.270042,0.188406,0.627451,0.717416,0.250000,0.477707,0.415810,0.666667,0.0,1.0
439,0.683333,0.270042,0.545942,0.318627,0.374856,0.263158,0.281847,0.305030,0.560606,0.0,1.0
440,0.283333,0.485232,0.478261,0.509804,0.482122,0.250000,0.444268,0.657026,0.409091,1.0,0.0


In [16]:
# Linear Regression
results = LinearRegression().fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
summary(results, X_scal.iloc[test_idx], Y.iloc[test_idx], xlabels=X_scal.columns)

Coefficients:
              Estimate  Std. Error  t value   p value
_intercept   -2.765884   14.019064  -0.1973  0.843901
AGE         -14.462769   23.423912  -0.6174  0.538011
BMI         127.144195   31.700337   4.0108  0.000101
BP           67.172560   28.006045   2.3985  0.017861
S1         -230.313267  163.737811  -1.4066  0.161897
S2          162.188278  116.061620   1.3974  0.164628
S3           22.483360   70.551823   0.3187  0.750474
S4           16.185916   56.936722   0.2843  0.776642
S5          207.514650   49.823271   4.1650  0.000056
S6           19.291168   33.881427   0.5694  0.570071
SEX_1        10.444984    7.062332   1.4790  0.141531
SEX_2       -10.444984    6.968529  -1.4989  0.136293
---
R-squared:  0.530165,    Adjusted R-squared:  0.487453,    MSE: 3084.6


In [18]:
penelty = [0.00001, 0.00005, 0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.6, 0.7, 0.9, 1, 10]

lasso_cv=LassoCV(alphas=penelty, cv=5)
model = lasso_cv.fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
print("Best Alpha:{0:.5f}, R2:None".format(model.alpha_))

Best Alpha:0.30000, R2:None


In [20]:
model_best = Lasso(alpha=model.alpha_).fit(X_scal.iloc[train_idx], Y.iloc[train_idx])

summary(model_best, X_scal.iloc[test_idx], Y.iloc[test_idx], xlabels=X_scal.columns)

Coefficients:
              Estimate  Std. Error  t value   p value
_intercept   35.476784   14.155053   2.5063  0.013415
AGE          -3.436299   23.651129  -0.1453  0.884703
BMI         127.199861   32.007837   3.9740  0.000116
BP           60.468162   28.277710   2.1384  0.034329
S1           -6.083815  165.326105  -0.0368  0.970701
S2           -0.000000  117.187444  -0.0000  1.000000
S3          -62.922734   71.236192  -0.8833  0.378682
S4            0.000000   57.489022   0.0000  1.000000
S5          124.200563   50.306568   2.4689  0.014832
S6            8.099838   34.210085   0.2368  0.813204
SEX_1        18.108153    7.130839   2.5394  0.012264
SEX_2        -0.000000    7.036125  -0.0000  1.000000
---
R-squared:  0.521006,    Adjusted R-squared:  0.477461,    MSE: 3144.7
